In [ ]:
!pip install -U scikit-learn

### Packages
import warnings
import itertools
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
import os

# Settings
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings("ignore")

In [ ]:
# Mount Google Drive for reading and writing files
drive.mount('/content/drive')
os.chdir("drive/My Drive/PROJECT/HealthCare/")

In [ ]:
def impute_missing_value(city_data):
    """
    Imputes 0 for first 12 months, 
    last year's value for months 12-24, 
    and minimum value of last two years for months 25+
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

def evaluate_forecast(y, pred):
    """Returns MAE, MSE, MAPE, and RMSE between y and pred"""
    results = pd.DataFrame({'r2_score':r2_score(y, pred),
                           }, index=[0])
    results['mae'] = mean_absolute_error(y, pred)
    # results['median_absolute_error'] = median_absolute_error(y, pred)
    results['mse'] = mean_squared_error(y, pred)
    # results['msle'] = mean_squared_log_error(y, pred)
    results['mape'] = mean_absolute_percentage_error(y, pred)
    results['rmse'] = np.sqrt(results['mse'])
    return results

In [ ]:
### SARIMA ###

# diarrhoea_cities = ['Điện Biên', 'Thái Bình', 'Lào Cai', 'Kon Tum', 'Cao Bằng']

# Load the data
vietnam = pd.read_excel("./data/full_data_fixed.xlsx")

# Restrict to training and validation sets unless doing final test evaluation
vietnam = vietnam.loc[vietnam['year_month'] < '2014-1-1']

# Select province/city for model
city = "Bình Phước"
dfbase = vietnam.loc[vietnam['province'] == city]

### Create data to fit
df = dfbase[['Dengue_fever_rates']]
df = impute_missing_value(df)

ddatefull = [pd.Timestamp(x) for x in list(dfbase.year_month)]
df['Date'] = ddatefull
df = df.set_index("Date")
df.dropna(inplace=True)

# number of sample
numdata = len(df)

# number of test
# last 3 years 2014-2016
numtest = 36

# divide into train and validation set
train = df[:(numdata - numtest)]
test = df[(numdata - numtest):]

# Indexing
start_index = test.index.min()
end_index = test.index.max()
start_pos = numdata - numtest
end_pos = numdata

# train and prediction
predictions = list()

trainlist = [x for x in train.Dengue_fever_rates]
testlist = [x for x in test.Dengue_fever_rates]

# prediction step
nstep = 1

# retrain the model
retrain = True

# set the model config
order = (4, 0, 6)
sorder = (6, 0, 3, 12)
trend = 'n'

# a first model to look at
model = SARIMAX(trainlist, order=order, seasonal_order=sorder, trend=trend,
                    enforce_stationarity=False,
                    enforce_invertibility=False)
# fit model
model_fit = model.fit(disp=False)

# step over each time-step in the test set
for i in range(len(testlist) - nstep + 1):
    print("Iteration ", i)
    if retrain:
        # define model
        model = SARIMAX(trainlist, order=order, seasonal_order=sorder, trend=trend,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        # fit model
        model_fit = model.fit(disp=False)
    # forecast
    yhat = model_fit.predict(len(trainlist), len(trainlist) + nstep - 1)
    # print the prediction
    # take the prediction result
    predictions.append(yhat[nstep - 1])
    # add actual observation to history for the next loop
    trainlist.append(testlist[i])

# print final model
print("Final model: ")
print(model_fit.summary())

# print out the prediction
# print("TrainList : ", trainlist)
# print("TestList : ", testlist)
# print("Prediction: ", predictions)

# prediction
predorg = df[(numdata - numtest + nstep - 1):].copy()
pred = df[(numdata - numtest + nstep - 1):].copy()
pred['Dengue_fever_rates'] = predictions

# evaluate forecast
results = evaluate_forecast(predorg['Dengue_fever_rates'], pred['Dengue_fever_rates'])
print(results)
print(f"P-values: {model_fit.pvalues}")

In [ ]:
# Save observed and predicted values, as well as error metrics

results_sarima = pd.DataFrame({'Observed':predorg['Dengue_fever_rates'], 
                                'SARIMA':pred['Dengue_fever_rates']})
results_sarima.reset_index(inplace=True)
results_sarima['RMSE'] = pd.Series(results.rmse)
results_sarima['MAE'] = pd.Series(results.mae)
results_sarima['MAPE'] = pd.Series(results.mape)

results_sarima.to_excel('df_binh_phuoc_SARIMA_1mo.xlsx')

In [ ]:

# SARIMAX

# diarrhoea_cities = ['Điện Biên', 'Thái Bình', 'Lào Cai', 'Kon Tum', 'Cao Bằng']

# Load the data
vietnam = pd.read_excel("./data/full_data_fixed.xlsx")

# Restrict to training and validation sets unless doing final test evaluation
vietnam = vietnam.loc[vietnam['year_month'] < '2014-1-1']

city = "Bình Phước"
dfbase = vietnam.loc[vietnam['province'] == city]

### Create data to fit
df = dfbase[['Dengue_fever_rates', 'Total_Evaporation', 'Max_Absolute_Temperature']]
df = impute_missing_value(df)

# Create lagged predictors
df['Total_Evaporation_lag1'] = df['Total_Evaporation'].shift(1)
df['Total_Evaporation_lag2'] = df['Total_Evaporation'].shift(2)
df['Total_Evaporation_lag3'] = df['Total_Evaporation'].shift(3)
df['Max_Absolute_Temperature_lag1'] = df['Max_Absolute_Temperature'].shift(1)
df['Max_Absolute_Temperature_lag2'] = df['Max_Absolute_Temperature'].shift(2)
df['Max_Absolute_Temperature_lag3'] = df['Max_Absolute_Temperature'].shift(3)

ddatefull = [pd.Timestamp(x) for x in list(dfbase.year_month)]
df['Date'] = ddatefull
df = df.set_index("Date")
df.dropna(inplace=True)

# Define prediction varible (y) and exogenous regressors (X)
y = df[['Dengue_fever_rates']]
X = df[['Total_Evaporation_lag1', 'Total_Evaporation_lag2', 
        'Total_Evaporation_lag3','Max_Absolute_Temperature_lag1', 
        'Max_Absolute_Temperature_lag2', 'Max_Absolute_Temperature_lag3']]

# number of sample
numdata = len(y)

# number of test
# last 3 years 2014-2016
numtest = 36

# divide into train and validation set
train_y, train_X = y[:(numdata - numtest)], X[:(numdata - numtest)]
test_y, test_X  = y[(numdata - numtest):], X[(numdata - numtest):]

# Create scale factor from exogenous training data
sc_in = MinMaxScaler(feature_range=(0, 1)).fit(train_X)

# Scale exogenous training and test data on training data scaler
train_X = sc_in.transform(train_X)
test_X = sc_in.transform(test_X)

# Rename columns
train_X = pd.DataFrame(train_X)
train_X.rename(columns={0: 'Total_Evaporation_lag1', 1: 'Total_Evaporation_lag2', 
                        2: 'Total_Evaporation_lag3', 3: 'Max_Absolute_Temperature_lag1', 
                        4: 'Max_Absolute_Temperature_lag2', 5: 'Max_Absolute_Temperature_lag3'}, inplace = True)
test_X = pd.DataFrame(test_X)
test_X.rename(columns={0: 'Total_Evaporation_lag1', 1: 'Total_Evaporation_lag2', 
                        2: 'Total_Evaporation_lag3', 3: 'Max_Absolute_Temperature_lag1', 
                        4: 'Max_Absolute_Temperature_lag2', 5: 'Max_Absolute_Temperature_lag3'}, inplace = True)

# Create scale factor from disease incidence training data
sc_out = MinMaxScaler(feature_range=(0, 1)).fit(train_y)

# Scale disease incidence training and test data on training data scaler
train_y = sc_out.transform(train_y)
test_y = sc_out.transform(test_y)

# Rename columns
train_y = pd.DataFrame(train_y)
train_y.rename(columns={0:"Dengue_fever_rates"}, inplace = True)
test_y = pd.DataFrame(test_y)
test_y.rename(columns={0:"Dengue_fever_rates"}, inplace = True)

# Indexing
start_index = test_y.index.min()
end_index = test_y.index.max()
start_pos = numdata - numtest
end_pos = numdata

# train and prediction
predictions = list()

trainlist_y = [x for x in train_y.Dengue_fever_rates]
testlist_y = [x for x in test_y.Dengue_fever_rates]
trainlist_X = train_X.values.tolist()
testlist_X = test_X.values.tolist()

# prediction step
nstep = 1

# retrain the model
retrain = True

# set the model config
order = (4, 0, 6)
sorder = (6, 0, 3, 12)
trend = 'n'

# a first model to look at
model = SARIMAX(trainlist_y, exog=train_X, order=order, seasonal_order=sorder, trend=trend,
                    enforce_stationarity=False,
                    enforce_invertibility=False)
# fit model
model_fit = model.fit(disp=False)

# step over each time-step in the test set
for i in range(len(testlist_y) - nstep + 1):
    print("Iteration ", i)
    if retrain:
        # define model
        model = SARIMAX(trainlist_y, exog=trainlist_X, order=order, seasonal_order=sorder, trend=trend,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        # fit model
        model_fit = model.fit(disp=False)
    # forecast
    yhat = model_fit.predict(len(trainlist_y), len(trainlist_y) + nstep - 1, exog=test_X[i:nstep + i])
    # print the prediction
    # take the prediction result
    predictions.append(yhat[nstep - 1])
    # add actual observation to history for the next loop
    trainlist_y.append(testlist_y[i])
    trainlist_X.append(testlist_X[i])

# print final model
print("Final model: ")
print(model_fit.summary())

# print out the prediction
# print("TrainList : ", trainlist)
# print("TestList : ", testlist)
# print("Prediction: ", predictions)

# prediction
predorg = df[(numdata - numtest + nstep - 1):].copy()
pred = df[(numdata - numtest + nstep - 1):].copy()
pred['Dengue_fever_rates'] = predictions

# unscale values
pred['Dengue_fever_rates'] = sc_out.inverse_transform(pred[['Dengue_fever_rates']])

# evaluate forecast
results = evaluate_forecast(predorg['Dengue_fever_rates'], pred['Dengue_fever_rates'])
print(results)
print(f"P-values: {model_fit.pvalues}")

In [ ]:
# Save observed and predicted values, as well as error metrics
results_sarimax = pd.DataFrame({'Observed':predorg['Dengue_fever_rates'], 
                                'SARIMAX':pred['Dengue_fever_rates']})
results_sarimax.reset_index(inplace=True)
results_sarimax['RMSE'] = pd.Series(results.rmse)
results_sarimax['MAE'] = pd.Series(results.mae)
results_sarimax['MAPE'] = pd.Series(results.mape)

results_sarimax.to_excel('df_binh_phuoc_SARIMAX_1mo.xlsx')